In [83]:
#Using Normal Openai 

import os
import openai
from dotenv import load_dotenv
load_dotenv('myenv/.env')
a=os.environ.get('OPENAI_API_KEY')
openai.api_key=a


In [84]:
#Simply loading the files 

from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader(
    input_files=["/Users/eshantdas/Desktop/Deeplearning_hehehe/Advance_lag/ebook.pdf"]
).load_data()

In [145]:
#Creating the Trulens Instance

from trulens_eval import Tru
tru = Tru()
tru.reset_database()

In [146]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.core.schema.Document'>
Doc ID: 7357a164-fefb-43cf-915c-fffede211589
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


In [87]:
import warnings
warnings.filterwarnings('ignore')

### Basic Rag Pipeline

In [88]:
from llama_index.core import Document
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [89]:

from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
# embed_model = AzureOpenAIEmbedding(
#     model="text-embedding-ada-002",
#     deployment_name="ada-embedding-model",
#     api_key='your_key',
#     azure_endpoint='your_endpoint',
#     api_version='2023-05-15'
# )
# llm = AzureOpenAI(
#     model="gpt-4-32k",
#     deployment_name="your_depployment_name",
#     api_key='your_key',
#     azure_endpoint='your_endpoint',
#     api_version='2023-05-15'
# )


In [90]:
load_dotenv('myenv/.env')
a=os.environ.get('OPENAI_API_KEY')
openai.api_key=a

In [91]:
#Initialising the Openai and OpenAIEmbedding

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
llm=OpenAI()
embed_model=OpenAIEmbedding()

In [92]:
#Storing the Index

from llama_index.core import VectorStoreIndex
from llama_index.core import ServiceContext
from llama_index.llms.openai import OpenAI

# llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [93]:
#Creating the query Engine

query_engine = index.as_query_engine()

In [94]:
#getting the response

response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

Develop a side hustle, ensure the project will help you grow technically, collaborate with good teammates, and consider if the project can be a stepping stone to larger projects.


### Evaluation setup using TruLens

In [95]:
#There is a txt file which has alot of questions and we will have that in the form of a list

eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [96]:
# You can try your own question :
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'What is the right AI job for me?']

In [97]:
from trulens_eval import Tru
tru = Tru()

# tru.reset_database()

In [98]:
#There is a utils file which I will be providing and it will contain the code to run the trulens recorder

from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [100]:
#Storing the responses as a recording in Trulens


with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [101]:


records, feedback = tru.get_records_and_feedback(app_ids=[])

In [102]:
#getting a view of the records which were stored in a dataframe
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_d041b0ee625a47f8b9648ac78033a73c,"""What are the keys to building a career in AI?""","""Keys to building a career in AI include learn...",-,"{""record_id"": ""record_hash_d041b0ee625a47f8b96...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-04-12T13:28:45.123838"", ""...",2024-04-12T13:28:48.597432,0.8,0.80,0.4625,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",3,1697,0.002559
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_ea5af86f4b767e4e05101de33a794ac3,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_ea5af86f4b767e4e051...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-04-12T13:28:48.814672"", ""...",2024-04-12T13:28:51.610424,0.9,0.10,0.9000,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,2,1683,0.002552
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_48fcea996b5b7cad6492b7669e70e579,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it helps in bu...",-,"{""record_id"": ""record_hash_48fcea996b5b7cad649...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-04-12T13:28:51.808760"", ""...",2024-04-12T13:28:54.459781,0.9,0.10,0.9250,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,2,1676,0.002538
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_5602f2ef94f1eb9553dd1f6e5d1cb08b,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_5602f2ef94f1eb9553d...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-04-12T13:28:54.665264"", ""...",2024-04-12T13:28:56.953762,1.0,0.80,1.0000,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Hopefully the previous c...,2,1643,0.002465
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_7d84cbf6b886dda1195fbca0554816c6,"""How can altruism be beneficial in building a ...","""Helping others during the journey of building...",-,"{""record_id"": ""record_hash_7d84cbf6b886dda1195...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-04-12T13:28:57.175890"", ""...",2024-04-12T13:28:59.041753,0.8,0.45,0.8250,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Hopefully the previous c...,1,1623,0.002427


In [103]:
type(records)

pandas.core.frame.DataFrame

In [104]:
import pandas as pd
records.to_csv('output.csv', index=False) 

In [105]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [106]:
# launches on http://localhost:8501/

#See the results in a streamlit app
tru.run_dashboard();

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.0.144:8501



### `Sentence Window Retrieval`

In [147]:
#THe codes below this block will be showing how you can run a query engine using Sentence Window Retrieval

In [107]:
from llama_index.llms.openai import OpenAI
llm = AzureOpenAI(
    model="gpt-4-32k",
    deployment_name="your_deployment_name",
    api_key='your_key',
    azure_endpoint='your_endpoint',
    api_version='2023-05-15'
)

In [129]:
llm=OpenAI()
embed_model=OpenAIEmbedding()

In [130]:
from utils import build_sentence_window_index
sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model=embed_model,
    save_dir="sentence_index",
)

In [131]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [132]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
# print(str(window_response))

In [133]:
# tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [134]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
The keys to building a career in AI involve learning foundational technical skills, working on projects, finding a job, and being part of a supportive community.
How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to collaborate effectively, influence one another, and be influenced by others. Working in teams on large projects in AI can lead to better outcomes compared to working individually. Interpersonal and communication skills play a crucial role in teamwork, as they enable team members to work together efficiently and share insights and expertise, ultimately enhancing the overall project success.
What is the importance of networking in AI?
Networking in AI is crucial as it not only provides valuable information but also opens up opportunities for referrals to potential employers. Building a network within the AI community can help individuals receive guidance, support, and ins

In [135]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Direct Query Engine,0.759903,0.881818,0.359091,2.090909,0.002850
Sentence Window Query Engine,0.686364,0.927273,0.481818,2.090909,0.000933


In [136]:
# launches on http://localhost:8501/

#Here you will see the difference between Sentence Window Query Engine and Normal QUery Engine how the performence varries
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.0.144:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## `Auto Merging Retrieval`

In [148]:
#THe codes below this block will be showing how you can run a query engine using Auto Merge Retrieval

In [138]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model=embed_model,
    save_dir="merging_index"
)

In [139]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [140]:
auto_merging_response =automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 3 nodes into parent node.
> Parent node id: e3045488-a49d-463a-938a-42c0d8b36eb3.
> Parent node text: Unfortunately, not every business problem can be 
reduced to optimizing test set accuracy! If you...

Building a portfolio of projects involves progressing from simple to complex undertakings over time. Communication is essential in explaining the value of your work to others, gaining their trust, and securing resources for larger projects. By effectively communicating your ideas and showcasing the progression of your projects, you can engage colleagues, mentors, and managers to support your initiatives.


In [141]:
# tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [142]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

What are the keys to building a career in AI?
The keys to building a career in AI are learning foundational technical skills, working on projects, finding a job, and being part of a community.
How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by fostering an environment where colleagues work hard, learn continuously, and collaborate to build AI that benefits all people. When individuals work together effectively, they can combine their skills and knowledge to tackle complex AI projects, leading to innovative solutions and successful outcomes. Additionally, being surrounded by motivated and dedicated colleagues can inspire individuals to strive for excellence and contribute positively to the team's overall success in AI endeavors.
What is the importance of networking in AI?
Networking in AI is crucial as it can provide valuable information, support, and potential job referrals. Building a strong professional network within the AI community can offer 

In [143]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Direct Query Engine,0.759903,0.881818,0.359091,2.090909,0.002850
Automerging Query Engine,0.745833,0.940000,0.588889,2.090909,0.000915
Sentence Window Query Engine,0.686364,0.927273,0.481818,2.090909,0.000933


In [144]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.0.144:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>